In [1]:
import os
import numpy as np
import pandas as pd
from astropy.table import Table
from bilby.hyper.model import Model
from gwpopulation.models.redshift import PowerLawRedshift
from tqdm import tqdm

from bilby.core.result import read_in_result

from gwpop.mass_models import (
    matter_matters_pairing,
    matter_matters_primary_secondary_independent,
)
from gwpop.spin_models import (
    iid_spin_magnitude_beta,
    iid_spin_orientation_gaussian_isotropic,
)
from pipe.gwpopulation_pipe_pdb import draw_true_values
from utils.cupy_utils import xp
from utils.map_utils import extract_map_parameters

/cvmfs/software.igwn.org/conda/envs/igwn-py39/lib/python3.9/site-packages/pandas/core/computation/expressions.py:21: UserWarning: Pandas requires version '2.8.4' or newer of 'numexpr' (version '2.7.3' currently installed).
  from pandas.core.computation.check import NUMEXPR_INSTALLED



#### ============================================================
#### 1. CONFIGURATION
#### ============================================================

In [2]:
RESULT_FILE = "data/baseline5_widesigmachi2_mass_NotchFilterBinnedPairingMassDistribution_redshift_powerlaw_mag_iid_spin_magnitude_gaussian_tilt_iid_spin_orientation_result.hdf5"
N_SAMPLES = int(1e6)
CHUNK_SIZE = int(1e3)
Z_MAX = 2.3


In [3]:
#### ============================================================
#### 2. LOAD MAP PARAMETERS FROM BILBY RESULT FILE
### ============================================================

In [4]:
# Load "Broken Power Law + 2 Peaks model" result
result = read_in_result(RESULT_FILE)
posterior = result.posterior.copy()

In [5]:

# Find the sample with maximum likelihood (same as max posterior for uniform priors)
max_likelihood_sample = posterior.loc[np.argmax(posterior.log_likelihood)]

In [6]:
# Set minimum and maximum allowed masses for the model (can be tuned)
max_likelihood_sample["absolute_mmin"] = 0.5
max_likelihood_sample["absolute_mmax"] = 350

In [9]:
print("Max parameters:")
print(max_likelihood_sample)

Max parameters:
A                  0.091462
A2                 0.828165
BHmax            152.055979
BHmin              7.763955
NSmax              4.094744
                    ...    
var_99             0.001657
variance           0.984675
xi_spin            0.712869
absolute_mmin      0.500000
absolute_mmax    350.000000
Name: 2411, Length: 334, dtype: float64


In [10]:
#### ============================================================
#### 3. If we need to replace alpha_1 and alpha_chi  values
#### ============================================================


In [11]:
# max_likelihood_sample['alpha_chi'] = 1
# max_likelihood_sample['alpha_1'] = 1

In [12]:
# Use absolute 
exclude_abs = {"alpha_1", "alpha_2", "log_prior"}
for key in max_likelihood_sample.index:
    if key not in exclude_abs:
        max_likelihood_sample[key] = np.abs(max_likelihood_sample[key])

In [ ]:
# ============================================================
# 4. CONSTRUCT POPULATION MODEL WITH MAP PARAMETERS
# ============================================================

In [ ]:
# Compose the model: mass, spin orientation, spin magnitude, redshift
model = Model([
    matter_matters_pairing,
    iid_spin_orientation_gaussian_isotropic,
    iid_spin_magnitude_beta,
    PowerLawRedshift(z_max=Z_MAX),
])


In [ ]:
# Set the parameters from MAP sample
model.parameters.update(max_likelihood_sample)

In [ ]:
#### ============================================================
#### 4. DRAW SYNTHETIC POPULATION SAMPLES (IN CHUNKS)
#### ============================================================

In [ ]:
n_chunks = int(np.ceil(N_SAMPLES / CHUNK_SIZE))
all_events = []

for i in tqdm(range(n_chunks), desc="Simulating events"):
    current_chunk_size = min(CHUNK_SIZE, N_SAMPLES - i * CHUNK_SIZE)
    events = draw_true_values(
        model=model,
        vt_model=None,
        n_samples=current_chunk_size,
        parameters=None,
    )
    #all_events.append(events)